In [1]:
import findspark
findspark.init("/home/jean/spark-2.4.4-bin-hadoop2.7")
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True, sep='\t')

In [5]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [6]:
data = data.withColumnRenamed('_c0', 'class')

In [7]:
data = data.withColumnRenamed('_c1','text')

In [8]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import length

In [10]:
data = data.withColumn('length', length(data['text']))

In [11]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [12]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [14]:
from pyspark.ml.feature import CountVectorizer, StopWordsRemover, Tokenizer, IDF, StringIndexer

In [15]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")

In [16]:
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')

In [17]:
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='c_vec')

In [18]:
idf = IDF(inputCol='c_vec', outputCol='tf_idf')

In [19]:
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [20]:
from pyspark.ml.feature import VectorAssembler

In [21]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol="features" )

In [22]:
from pyspark.ml.classification import NaiveBayes

In [23]:
nb = NaiveBayes()

In [24]:
from pyspark.ml import Pipeline

In [30]:
pipeline = Pipeline(stages=[ham_spam_to_numeric,tokenizer, stop_remove, count_vec, idf, clean_up])

In [31]:
cleaner = pipeline.fit(data)

In [32]:
clean_data = cleaner.transform(data)

In [33]:
clean_data.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,8

In [34]:
clean_data = clean_data.select(["label", "features"])

In [35]:
train_data, test_data = clean_data.randomSplit([0.7,0.3])

In [36]:
spam_detector = nb.fit(train_data)

In [37]:
test_results = spam_detector.transform(test_data)

In [38]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-607.41893871406...|[1.0,2.2120915098...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-574.85746552971...|[0.99999999999651...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-999.49570639648...|[1.0,2.9802579478...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-869.29677528421...|[1.0,2.8329812226...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-886.92290330018...|[1.0,5.4983420601...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-218.42403951577...|[1.0,6.7712460110...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-691.14578723088...|[1.0,4.6247950756...|       0.0|
|  0.0|(13424,[0,1,17,19...|[-803.53293752032...|[1.0,4.4498744035...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-756.44922312112...|[1.0,5.4307850011...|       0.0|
|  0.0|(13424,[0

In [39]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [40]:
acc_eval = MulticlassClassificationEvaluator()

In [41]:
acc_eval.evaluate(test_results)

0.9160002014334981